## 1) Import des modules :

In [ ]:
from pymongo import MongoClient
import json

## 2) Création du client 

In [ ]:
# Création de l'objet client avec le port :
client = MongoClient('localhost', 27017)
db = client.DBLP.publis

## 3) Ajout des documents en base à partir du JSON :

In [ ]:
chemin = 'dblp.json'


def ajout_documents_en_base (chemin) :
    
    # Création de la liste de documents à partir du fichier json :
    liste_dblp = list(open(chemin, 'r'))

    percent_count = 0
    absolute_count = 0
    error_count = 0

    # On boucle sur la liste :
    for line in liste_dblp:
        #On essaie d'insérer la ligne dans la collection publis de la base DBLP crées implicitement :
        try:
            client.DBLP.publis.insert_one(json.loads(line))
        except:
            # On incrémente le compteur d'erreur si la ligne n'a pas pu être ajoutée :
            error_count += 1
            print(error_count, "error(s)")
            pass
        absolute_count +=1

        # Comptage du nombre de documents ajoutés :
        if int((absolute_count / len(liste_dblp)) * 100) > percent_count:
            percent_count += 1
            print(percent_count, "/", len(liste_dblp))

    print("Insertion terminée.")
    
ajout_documents_en_base(chemin)

## 4) Fonctions demandées :

Requêtes par script demandées dans le brief

In [ ]:
# 1) Compter nombre de docs :
# Forme dépréciée :
#client.DBLP.publis.find({}).count()

# Forme conseillée
def tous_documents():
    resultat = db.count_documents({})
    print("Il y a", resultat, "documents en base.")

tous_documents()

In [ ]:
# 2) Lister tous les livres (type “Book”) :
def tous_types_book():
    resultat = db.find({"type" : "Book"}, {"_id" : 0})

    for r in resultat:
        print(r)
        print()
        
tous_types_book()

In [ ]:
# 3) Lister les livres depuis 2014 :
def tous_livres_depuis(annee):
    resultat = db.find({"year" : {"$gte" : annee}}, {"_id" : 0})

    for r in resultat:
        print(r)
        print()

tous_livres_depuis(2014)

In [ ]:
# 4) Liste des publications de Toru Ishida
def toutes_publis_auteur(auteur):
    resultat = db.find({"authors": auteur}, {"_id" : 0})
    for r in resultat:
        print(r)
        print()
        
toutes_publis_auteur("Toru Ishida")

In [ ]:
# 5) Liste des auteurs distincts
def tous_auteurs():
    resultat = db.distinct("authors")
    for r in resultat:
        print(r)
        print()
        
tous_auteurs()

In [ ]:
# 6) Trier les publications de Toru Ishida par titre de livre
def publis_par_auteur_triees(auteur):
    
    resultat = db.find({"$query": {"authors": auteur}, "$orderby": {"title": 1}},{"_id" : 0})
    for r in resultat:
        print(r)
        print()
        
        # Pour utiliser la fonction :
        #publis_par_auteur_triees("Toru Ishida") 
        
publis_par_auteur_triees("Toru Ishida")

In [ ]:
#7)  Compter le nombre de  publication de Toru Ishida :
def nb_publis_auteur(auteur):
    resultat = db.count_documents({"authors":auteur})
    print("Toru Ishida a fait",resultat, "publications")
nb_publis_auteur("Toru Ishida")

In [ ]:
#8)  Compter le nombre de publications depuis 2011 et par type :
def nb_publis_depuis_annee (annee):
    compteuranneetype =  db.find({"year":{"$gte":annee}})
    liste_types  = compteuranneetype.distinct( "type" )

    for type in liste_types :

        print("Depuis 2011,",db.count_documents({"type" : type}),type+"(s)","ont été publiés")

nb_publis_depuis_annee(2011)

In [ ]:
    
#9) Compter le nombre de publications par auteur et trier le résultat par ordre croissant :
def nb_publis_par_auteur ():
    x = db.aggregate([{ "$unwind" : "$authors" },
                      {"$group": { "_id" : "$authors", "total": { "$sum": 1 } } },
                      {"$sort": {"total":1} }
                      ])

    for i in x:
        print(i)

nb_publis_par_auteur ()

## 4) Script facultatif avec demande de chemin et insertion JSON

In [ ]:
def ajout_documents_en_base_V2() :
    
    chemin_ok = False
    
    while chemin_ok != True :
        chemin = str(input("Veuillez insérez le chemin vers le fichier JSON à importer."))

        if chemin.isnumeric() == False:

            try :
                # Création de la liste de documents à partir du fichier json :
                liste_dblp = list(open(chemin, 'r'))
                chemin_ok = True

            except :
                print("chemin invalide")

        else : 
            print("Veuillez entrer un chemin valide vers un fichier JSON")

    # Création de l'objet client avec le port :
    client = MongoClient('localhost', 27017)
    
    percent_count = 0
    absolute_count = 0
    error_count = 0

    test_collec = input("Entre le nom de la collection à remplir")

    # On boucle sur la liste :
    for line in liste_dblp:
        # Dans la database DBLP créée implicitement, on essaie d'insérer la ligne dans la collection
        # dont le nom est dans la var test_collec.
        try:
            client.DBLP[test_collec].insert_one(json.loads(line))
        except:
            # On incrémente le compteur d'erreur si la ligne n'a pas pu être ajoutée :
            error_count += 1
            print(error_count, "error(s)")
            pass
        absolute_count +=1

        # Comptage du nombre de documents ajoutés :
        if int((absolute_count / len(liste_dblp)) * 100) > percent_count:
            percent_count += 1
            print(percent_count, "/", len(liste_dblp))
            

    print("Insertion terminée.")

ajout_documents_en_base_V2()